In [5]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from   scipy.integrate import solve_ivp as ode45
import numpy as np

%matplotlib widget
plt.ioff()

def plot_sets(ax,gr=False,ti='',xla=False,yla=False,zla=False,xli=False,yli=False,zli=False,ticks_off=False,xticks_off=False,yticks_off=False,zticks_off=False, ba=False, ar=False,leg=False,view=False):
    
    ax.set_title(ti)
    if gr:  ax.grid(gr);ax.set_axisbelow(True)
    if xla: ax.set_xlabel(xla)
    if yla: ax.set_ylabel(yla)
    if zla: ax.set_zlabel(zla)

    if xticks_off: ax.set_xticklabels([])
    if yticks_off: ax.set_yticklabels([])
    if zticks_off: ax.set_zticklabels([])

    if   ticks_off==1: ax.set_xticklabels([]); ax.set_xticks([]);ax.set_yticklabels([]); ax.set_yticks([])
    elif ticks_off==2: ax.set_xticklabels([])
    elif ticks_off==3: ax.set_yticklabels([])

    if ba: ax.set_box_aspect(ba)
    if ar: ax.set_aspect(ar)

    if xli: ax.set_xlim(xli)
    if yli: ax.set_ylim(yli)
    if zli: ax.set_zlim(zli)

    if leg: ax.legend(**leg)

    if view:ax.view_init(**view)
 
px2inch = 1/plt.rcParams['figure.dpi']

## Custom styles
##################
plt.rcdefaults()

FS = 14

plt.rc('axes'  ,titlesize=FS) # title
plt.rc('axes'  ,labelsize=FS+1) # xy-labels font
plt.rc('grid'  ,linewidth=0.5) 
plt.rc('grid'  ,alpha=0.5)
plt.rc('grid'  ,linestyle='-.') 

#plt.rc('axes'  ,facecolor='whitesmoke')
plt.rc('xtick' ,labelsize=FS) # x-ticks font
plt.rc('ytick' ,labelsize=FS) # x-ticks font 

plt.rc('legend',fontsize =FS-1) # legend font
plt.rc('legend',facecolor='w')
plt.rc('lines' ,linewidth=0.75)    #  font

plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": "Times New Roman"
})

In [6]:
b = 0.1

def f(x):
    return 1/4 * x**4 - 1/2*x**2

def dfdt(t,y):

    return np.array([ [ y[1]               ],
                      [ y[0]-y[0]**3-b*y[1]], ]).flatten()

In [7]:
N     = 20
lim   = 1.85

tf    = 50
t     = np.linspace(0,tf,1000)
x     = np.linspace(-lim,lim,10*N)

y01    = np.array([-1.7,0])
y02    = np.array([-1.7,1])

def get_data(y0):

        # Potencial function P
        P = f(x)

        # Phase Space, compute vector field Sx,Sy,Su,Sv
        k     = 0.1
        sxy   = np.linspace(-lim+k,lim-k,N)
        Sx,Sy = np.meshgrid(sxy,sxy)
        Y     = np.zeros((2,N**2))

        for i,(sx,sy) in enumerate( zip(Sx.flatten(),Sy.flatten()) ):
                y      = np.array([sx,sy])
                Y[:,i] = dfdt(t,y)

        Y  = Y / np.sqrt( np.sum(Y**2,axis=0) )
        Su = Y[0,:].reshape((N,N))
        Sv = Y[1,:].reshape((N,N))

        # Integrate the dfdt(t,y)
        sol   = ode45(dfdt,(0,tf),y0,t_eval=t)
        phase = sol.y
        px = phase[0,:]
        py = phase[1,:]

        return P,Sx,Sy,Su,Sv,px,py

In [8]:
aspect_ratio = 1
cm2in        = 1/2.54
figw         = 19*cm2in
figh         = figw*aspect_ratio**-1 
fig, ax      = plt.subplots(3,1,constrained_layout=True,figsize=(figw,figh),sharex=True)
ax           = ax.flatten()

fig, ax      = plt.subplot_mosaic([[0,0,3,3],
                                   [1,1,4,4],
                                   [1,1,4,4],
                                   [2,2,5,5]],
                                   layout='constrained',figsize=(figw,figh),sharex=True)

c_potencial = 'tab:green' 
c_lines     = 'tab:blue' 

y0 = y01
P,Sx,Sy,Su,Sv,px,py = get_data(y0)

axi=ax[0]
axi.plot(x,P,c_potencial)
s1a = axi.scatter(y0[0],f(y0[0]),zorder=100,c=c_lines)
plot_sets(axi,gr=True,ti='',yla='Potencial',xli=[-lim,lim],ba=0.5)

axi=ax[1]
axi.quiver(Sx,Sy,Su,Sv,color=c_potencial,zorder=100,headwidth=6)
axi.plot(px,py,c_lines,zorder=50)
s1b=axi.scatter(px[0],py[0],zorder=100,c=c_lines)
plot_sets(axi,gr=True,ti='',yla='Velocity',xli=[-lim,lim],ba=1)

axi=ax[2]
axi.plot(px,t,c=c_lines)
s1c=axi.scatter(px[0],t[0],zorder=100,c=c_lines)
plot_sets(axi,gr=True,ti='',xla='X',yla='Time',xli=[-lim,lim],ba=0.5)

c_lines= 'tab:red' 
y0 = y02
P,Sx,Sy,Su,Sv,px,py = get_data(y0)

axi=ax[3]
axi.plot(x,P,c_potencial)
s2a=axi.scatter(y0[0],f(y0[0]),zorder=100,c=c_lines)
plot_sets(axi,gr=True,ti='',xli=[-lim,lim],ba=0.5,ticks_off=3)

axi=ax[4]
axi.quiver(Sx,Sy,Su,Sv,color=c_potencial,zorder=10,headwidth=6)
axi.plot(px,py,c_lines,zorder=50)
s2b=axi.scatter(px[0],py[0],zorder=100,c=c_lines)
plot_sets(axi,gr=True,ti='',xli=[-lim,lim],ba=1,ticks_off=3)

axi=ax[5]
axi.plot(px,t,c=c_lines)
s2c=axi.scatter(px[0],t[0],zorder=100,c=c_lines)
axi.set_xticks(np.arange(-1.5,1.75,0.5))
plot_sets(axi,gr=True,ti='',xla='X',xli=[-lim,lim],ba=0.5,ticks_off=3)


def animate_fun(i):
    
 
    s2a.set_offsets(( px[i], f(px[i]) ))
    s2b.set_offsets(( px[i], py[i]    ))
    s2c.set_offsets(( px[i], t[i]     ))

    
 
nframes = 1000
ani     = animation.FuncAnimation(fig=fig,
                                 func=animate_fun,
                                 frames=nframes)

writer = animation.PillowWriter(fps=10,
                                metadata=dict(artist='Me'),
                                bitrate=-1)
# ani.save('imgs/animation_subplot.gif', writer=writer)

#plt.show()

# plt.savefig('imgs/PotencialWell.pdf', dpi=300)
# plt.show()

In [ ]:
1000/25

40.0

40.0